In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import math
import time
import os
import random
import random
import re
import unicodedata
from tqdm import tqdm

2025-11-14 16:31:23.530518: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763137883.738580      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763137883.795155      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers
from tokenizers.trainers import BpeTrainer

In [3]:
# Config
seed = 1234
DATA_DIR = 'data'
SEQ_LEN = 100
VOCAB_SIZE = 5000
EMB_DIM = 256
ENC_HIDDEN_DIM = 512
DEC_HIDDEN_DIM = 512
DROPOUT = 0.3
N_EPOCHS = 200
LEARNING_RATE = 1e-2
BATCH_SIZE = 64
# Output directories
CHECKPOINT_DIR = 'checkpoints'
LOG_DIR = 'runs'
tokenizer_path = "tknzer_dir"
special_tokens = ["<pad>", "<st>", "<end>", "<unk>"]
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
seed_everything(seed)

In [5]:
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

In [6]:
def load_txt(txt_file):
    with open(txt_file, "r", encoding="utf-8") as f:
        text = f.read()
    return text

In [7]:
dataset_path = "/kaggle/input/iiith-assignment2-dataset/dataset/Pride_and_Prejudice-Jane_Austen.txt"

In [8]:
def normalize_text(text, lowercase = True):
    if lowercase:
        text = text.lower()
    # Unicode Normalization (e.g., converting fancy quotes to standard ones)
    text = unicodedata.normalize('NFKC', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[\*_`\[\]\^{}]', '', text) # handle _word_ and *word*
    text = re.sub(r'\b(mr|mrs|ms|dr|st)\.', r'\1', text, flags=re.IGNORECASE) # handle mr. mrs. etc
    text = text.strip()
    return text

In [9]:
custom_dataset = normalize_text(load_txt(dataset_path))

In [10]:
len(set(custom_dataset)), len(list(custom_dataset))

(56, 702635)

In [11]:
def get_tokenizer(dataset, vocab_size, save_fldr, name = 'bpe'):
    if os.path.exists(save_fldr):
        save_path = os.path.join(save_fldr, 'tokenizer.json')
        return Tokenizer.from_file(save_path)
    if name == 'bpe':
        tokenizer = Tokenizer(models.BPE())
        tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
        trainer = BpeTrainer(vocab_size=vocab_size, special_tokens=special_tokens)
        tokenizer.train_from_iterator([dataset], trainer)
        if save_fldr:
            os.makedirs(save_fldr, exist_ok=True)
            tokenizer.save(os.path.join(save_fldr, 'tokenizer.json'))
            return tokenizer

In [12]:
bpe_tokenizer = get_tokenizer(custom_dataset, VOCAB_SIZE, tokenizer_path, 'bpe')

In [13]:
tokenized_data = bpe_tokenizer.encode(custom_dataset).ids

In [14]:
len(tokenized_data)

154018

In [15]:
def get_data(data, split, train_percent = 0.8):
    n = len(data)
    split_percent = train_percent
    if split == "train":
        return data[ : int((split_percent)*n)]
    elif split == "val":
        return data[int((split_percent)*n) : ]
    else:
        return None

In [16]:
train_data = get_data(tokenized_data,'train', train_percent = 0.8)
val_data = get_data(tokenized_data,'val', train_percent = 0.8)

In [17]:
len(train_data)

123214

In [18]:
# Special token ids
PAD_TOKEN = bpe_tokenizer.token_to_id('<pad>')
ST_TOKEN = bpe_tokenizer.token_to_id('<st>')
END_TOKEN = bpe_tokenizer.token_to_id('<end>')
FULL_STOP = bpe_tokenizer.token_to_id('.')
QUESTION_TOKEN = bpe_tokenizer.token_to_id('?')
EXCLAMATION = bpe_tokenizer.token_to_id('!')
special_ids = PAD_TOKEN, ST_TOKEN, END_TOKEN, FULL_STOP, QUESTION_TOKEN, EXCLAMATION
special_ids

(0, 1, 2, 15, 29, 4)

In [19]:
# Dataset
class SimpleDataset(Dataset):
    def __init__(self, data, special_ids, seq_len):
        super().__init__()
        self.token_ids = data
        self.seq_len = seq_len
        self.pad, self.st, self.end, self.full_stop, self.quest, self.excl= special_ids
        self.sentence_enders = {self.full_stop, self.quest, self.excl}
        
        self.inputs = []
        self.decoder_inputs = []
        self.decoder_targets = []
        
        
        self.create_sequences()
        
    def create_sequences(self):
        tokens = self.token_ids
        n = len(tokens)
        L = self.seq_len
        
        i = 0
        
        while i < n-1:
            enc_seq = []
            seq_end_idx = i 
            for j in range(L):
                current_idx = i + j
                if current_idx >= n:
                    break
                token = tokens[current_idx]
                
                enc_seq.append(token)
                seq_end_idx = current_idx
                
                if token in self.sentence_enders:
                    break
            
            
            if not enc_seq or len(enc_seq) < 2:
                i = seq_end_idx + 1 # Move to the next token
                continue

            dec_target_seq = enc_seq[1:] + [self.end]
            dec_input_seq = [self.st] + dec_target_seq[:-1]
                
            # Pad sequences if shorter
            enc_seq_len = len(enc_seq)
            enc_seq = enc_seq + [self.pad] * max(L - enc_seq_len, 0)
            dec_input_seq = dec_input_seq + [self.pad] * max(L - enc_seq_len, 0)
            dec_target_seq = dec_target_seq + [self.pad] * max(L - enc_seq_len, 0)

            self.inputs.append(enc_seq)
            self.decoder_inputs.append(dec_input_seq)
            self.decoder_targets.append(dec_target_seq)
            
            i = seq_end_idx + 1
    
    def __len__(self):
        return len(self.inputs)
    
    
    def __getitem__(self, idx):
        return (
            torch.tensor(self.inputs[idx], dtype=torch.long), # encoder input 50 timesteps
            torch.tensor(self.decoder_inputs[idx], dtype=torch.long), # decoder inputs shifter from target 50 steps
            torch.tensor(self.decoder_targets[idx], dtype=torch.long) # decoder targets 50 steps
        )

In [20]:
train_dataset = SimpleDataset(train_data, special_ids, SEQ_LEN)
val_dataset = SimpleDataset(val_data, special_ids, SEQ_LEN)

In [21]:
len(train_dataset), len(val_dataset)

(3667, 1225)

In [22]:
# Example
val_dataset[0]

(tensor([ 370, 2784,   13,  406,  876,   15,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]),
 tensor([   1, 2784,   13,  406,  876,   15,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,

In [23]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [24]:
%%writefile model.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

# ENCODER
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, dropout=0.1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.enc_hidden_dim = hidden_dim
        self.lstm = nn.LSTM(
            input_size=emb_dim,
            hidden_size=hidden_dim,
            num_layers=2,
            batch_first=True,
            bidirectional=True,
            dropout=dropout
        )
    
    def forward(self, x):
        # x: [batch, seq_len]
        emb = self.embedding(x)  # [batch, seq_len, emb_dim]
        outputs, (h, c) = self.lstm(emb)  # outputs: [batch, seq_len, 2*hidden]
        return outputs, (h, c)

# ATTENTION 
class Attention(nn.Module):
    def __init__(self, enc_hidden_dim, dec_hidden_dim):
        super().__init__()
        
        # We need to align the dimensions of encoder outputs and decoder hidden state
        # enc_hidden_dim is for one direction, but encoder is bidirectional (2*)
        self.attn = nn.Linear((enc_hidden_dim * 2) + dec_hidden_dim, dec_hidden_dim)
        self.v = nn.Linear(dec_hidden_dim, 1, bias = False)
        
    def forward(self, dec_hidden, enc_outputs):
        # dec_hidden: [batch, dec_hidden_dim] (from the *top layer* of decoder)
        # enc_outputs: [batch, src_len, enc_hidden_dim * 2]
        
        batch_size = enc_outputs.shape[0]
        src_len = enc_outputs.shape[1]
        
        # Repeat decoder hidden state src_len times to concatenate
        # dec_hidden: [batch, src_len, dec_hidden_dim]
        dec_hidden = dec_hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        # energy: [batch, src_len, (enc_hidden * 2) + dec_hidden]
        energy_input = torch.cat((dec_hidden, enc_outputs), dim = 2)
        
        # energy: [batch, src_len, dec_hidden_dim]
        energy = torch.tanh(self.attn(energy_input))
        
        # v(energy): [batch, src_len, 1] -> [batch, src_len]
        attention = self.v(energy).squeeze(2)
        
        # Return softmax'd weights
        return F.softmax(attention, dim=1)

# Decoder
class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, enc_hidden_dim, dec_hidden_dim, dropout=0.1):
        super().__init__()
        self.dec_hidden_dim = dec_hidden_dim
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        
        self.attention = Attention(enc_hidden_dim, dec_hidden_dim)
        
        self.lstm = nn.LSTM(
            input_size=emb_dim + (enc_hidden_dim * 2),
            hidden_size=dec_hidden_dim,
            num_layers=2,
            batch_first=True,
            dropout=dropout
        )
        
        self.fc = nn.Linear(dec_hidden_dim + (enc_hidden_dim * 2), vocab_size)
    
    def forward(self, dec_input, dec_hidden, enc_outputs):
        # dec_input: [batch] -> current token IDs
        # dec_hidden: (h, c) from previous step
        # enc_outputs: [batch, src_len, 2*enc_hidden_dim]
        
        # dec_input [batch] -> [batch, 1]
        dec_input = dec_input.unsqueeze(1)
        
        # emb: [batch, 1, emb_dim]
        emb = self.embedding(dec_input)
        
       
        a = self.attention(dec_hidden[0][-1], enc_outputs)
        
        a = a.unsqueeze(1)
        
        context = torch.bmm(a, enc_outputs)
        
        lstm_input = torch.cat([emb, context], dim=2)
        output, dec_hidden = self.lstm(lstm_input, dec_hidden)
        
        output = output.squeeze(1)
        context = context.squeeze(1)

        concat_output = torch.cat([output, context], dim=1)
        
        logits = self.fc(concat_output)
        
        return logits, dec_hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        self.enc_num_layers = self.encoder.lstm.num_layers
        self.enc_num_directions = 2 if self.encoder.lstm.bidirectional else 1
        self.enc_hidden_dim = self.encoder.enc_hidden_dim
        self.dec_hidden_dim = self.decoder.dec_hidden_dim
        
        self.fc_hidden = nn.Linear(self.enc_hidden_dim * self.enc_num_directions, self.dec_hidden_dim)
        self.fc_cell = nn.Linear(self.enc_hidden_dim * self.enc_num_directions, self.dec_hidden_dim)

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.fc.out_features
        
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)
        
        enc_outputs, (h, c) = self.encoder(src)
        
        h = h.view(self.enc_num_layers, self.enc_num_directions, batch_size, self.enc_hidden_dim)
        c = c.view(self.enc_num_layers, self.enc_num_directions, batch_size, self.enc_hidden_dim)
        
        h_cat = torch.cat((h[:, 0, :, :], h[:, 1, :, :]), dim=2)
        c_cat = torch.cat((c[:, 0, :, :], c[:, 1, :, :]), dim=2)
        
        dec_hidden = (torch.tanh(self.fc_hidden(h_cat)), 
                      torch.tanh(self.fc_cell(c_cat)))
        
        dec_input = trg[:, 0]
        
        # Loop from 0, store at t
        for t in range(trg_len):
            
            # The decoder's forward pass now includes the attention mechanism
            logits, dec_hidden = self.decoder(dec_input, dec_hidden, enc_outputs)
            
            outputs[:, t] = logits
            
            use_teacher_force = random.random() < teacher_forcing_ratio
            
            if use_teacher_force:
                if t < trg_len - 1:
                    dec_input = trg[:, t+1]
                else:
                    break
            else:
                top1 = logits.argmax(1)
                dec_input = top1
            
        return outputs

Writing model.py


In [25]:
from model import Encoder, Decoder, Seq2Seq

In [26]:
encoder = Encoder(VOCAB_SIZE, EMB_DIM, ENC_HIDDEN_DIM, dropout=DROPOUT)
decoder = Decoder(VOCAB_SIZE, EMB_DIM, ENC_HIDDEN_DIM, DEC_HIDDEN_DIM, dropout=DROPOUT)
model = Seq2Seq(encoder, decoder, DEVICE).to(DEVICE)

In [27]:
model

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(5000, 256)
    (lstm): LSTM(256, 512, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(5000, 256)
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (lstm): LSTM(1280, 512, num_layers=2, batch_first=True, dropout=0.3)
    (fc): Linear(in_features=1536, out_features=5000, bias=True)
  )
  (fc_hidden): Linear(in_features=1024, out_features=512, bias=True)
  (fc_cell): Linear(in_features=1024, out_features=512, bias=True)
)

In [28]:
def load_ckpt(model):
    if os.path.exists(CHECKPOINT_DIR):
        models = [os.path.join(CHECKPOINT_DIR,ckpt) for ckpt in os.listdir(CHECKPOINT_DIR)]
        if len(models) > 0:
            sorted_models = sorted(models, key=lambda x: x.split("_val")[1].split('.pt')[0])
            model.load_state_dict(torch.load(sorted_models[0], map_location=DEVICE))
            print(f"Loaded checkpoint from {sorted_models[0]}")
            best_loss = float(os.path.basename(sorted_models[0]).split("_val")[1].split('.pt')[0])
            last_epoch = int(os.path.basename(sorted_models[0]).split("_val")[0].split('Epoch')[1])
            return best_loss, last_epoch
    

In [29]:
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.2, patience=5)

In [30]:
criterion = nn.CrossEntropyLoss(ignore_index=PAD_TOKEN)

In [31]:
def train(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0
    for i, (src, dec_in, trg) in enumerate(iterator):
        
        src, dec_in, trg = src.to(DEVICE), dec_in.to(DEVICE), trg.to(DEVICE)
        
        optimizer.zero_grad()
        output = model(src, dec_in)
        
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)
        trg = trg[:, 1:].reshape(-1)
        
        loss = criterion(output, trg)
        loss.backward()
        
        optimizer.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [32]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, (src, dec_in, trg) in enumerate(iterator):
            src, dec_in, trg = src.to(DEVICE), dec_in.to(DEVICE), trg.to(DEVICE)
            output = model(src, dec_in)
            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)
            trg = trg[:, 1:].reshape(-1)
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)

In [33]:
def generate_seq(model, texts, tokenizer, seq_len, special_ids, gen_len=100):
    model.eval()
    samples = []
    pad, st, end, full_stop, quest, excl= special_ids

    with torch.no_grad():
        for text in texts:
            # prepare encoder input
            text = normalize_text(text)
            ids = tokenizer.encode(text).ids
            if len(ids) < seq_len:
                ids = ids + [pad] * (seq_len - len(ids))
            else:
                print("Warning: More than seq length -- considering first 50 tokens:")
                ids = ids[:seq_len]

            src = torch.tensor([ids], dtype=torch.long, device=DEVICE)
            # print(src.shape)
            # print(src)

            # start decoder with only the start token (length = 1)
            dec_in = torch.tensor([[st]], dtype=torch.long, device=DEVICE)

            generated = []
            for step in range(gen_len):
                out = model(src, dec_in)          # [1, cur_dec_len, vocab_size]
                next_token = int(out[0, -1].argmax().cpu().item())  # last timestep prediction
                generated.append(next_token)
                
                # if next_token == end:
                #     break

                

                # append predicted token to decoder input for next step
                dec_in = torch.cat(
                    [dec_in, torch.tensor([[next_token]], dtype=torch.long, device=DEVICE)],
                    dim=1
                )
            # convert ids -> tokens
            # print("Out Tokens", generated)
            tokens = [tokenizer.id_to_token(tid) for tid in generated]
            samples.append((text, tokens))

    print("=== Sample generations ===")
    for idx, (inp, toks) in enumerate(samples, 1):
        print(f"[{idx}] INPUT : {inp}")
        print(f"OUTPUT: {' '.join(toks)}")
    print("======================================================")
    return samples

In [34]:
writer = SummaryWriter(LOG_DIR)

In [35]:
# text from internet
"""If I could have ceased what pendulums swung, or wheels turned, 
or water clocks emptied, then, in order to keep the Fates from marching in time,
I would have, for though it is what a boy naturally wishes when he fears change will 
come upon what he loves and take it away, a man remembers it, too, and in his heart 
wishes the same when all around him he feels only loss, loss that has been his 
companion for some time, and promises to remain at his side."""


"""
It is quite an interesting format, if you don't like the opening you are reading,
then you can skip and go to the next book opening. If the opening does capture your attention
you can click at the bottom to reveal the title and author. It reminds me of blind date books 
where bookstores will wrap a random book up in paper (to prevent the title and author from being seen)
and will write brief descriptions about it.


"""

"\nIt is quite an interesting format, if you don't like the opening you are reading,\nthen you can skip and go to the next book opening. If the opening does capture your attention\nyou can click at the bottom to reveal the title and author. It reminds me of blind date books \nwhere bookstores will wrap a random book up in paper (to prevent the title and author from being seen)\nand will write brief descriptions about it.\n\n\n"

In [36]:
CUSTOM_TEXTS = [
                "If I could have ceased",
                "It is quite an interesting format, if you"
        ]

In [37]:
def run_lm(predict_while_train = True, resume_ckpt = True):
    best_valid_loss = float('inf')
    last_epoch = 0
    if resume_ckpt:
        if os.listdir(CHECKPOINT_DIR):
            best_valid_loss, last_epoch = load_ckpt(model)
    
    for epoch in tqdm(range(N_EPOCHS), desc="Started Training"):
        epoch += last_epoch
        start_time = time.time()
        
        if predict_while_train:
            if epoch%5 == 0:
                generate_seq(model, CUSTOM_TEXTS, bpe_tokenizer, SEQ_LEN, special_ids, gen_len=50)

        train_loss = train(model, train_loader, optimizer, criterion)
        valid_loss = evaluate(model, val_loader, criterion)

        scheduler.step(valid_loss)
        
        end_time = time.time()
        
        epoch_mins = int((end_time - start_time) / 60)
        epoch_secs = int((end_time - start_time) - (epoch_mins * 60))

        # Save checkpoint if validation loss has improved
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), os.path.join(CHECKPOINT_DIR, f'Epoch{epoch}_val{valid_loss}.pt'))
            print(f"Checkpoint saved: New best validation loss {best_valid_loss:.3f}")

        # TensorBoard logging
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Perplexity/train', math.exp(train_loss), epoch)
        writer.add_scalar('Loss/validation', valid_loss, epoch)
        writer.add_scalar('Perplexity/validation', math.exp(valid_loss), epoch)
        writer.add_scalar('learning_rate', optimizer.param_groups[0]['lr'], epoch)

        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

In [38]:
run_lm(predict_while_train = True, resume_ckpt = True)

Started Training:   0%|          | 0/200 [00:00<?, ?it/s]

=== Sample generations ===
[1] INPUT : if i could have ceased
OUTPUT: attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra attra
[2] INPUT : it is quite an interesting format, if you
OUTPUT: attra attra attra attra attra attra attra because attra attra attra attra attra attra attra attra fa attra attra attra attra attra attra attra attra fa attra attra attra attra attra attra attra attra fa attra attra attra attra attra attra attra attra fa attra attra attra attra attra attra


Started Training:   0%|          | 1/200 [01:05<3:36:57, 65.41s/it]

Checkpoint saved: New best validation loss 6.211
Epoch: 01 | Time: 1m 5s
	Train Loss: 6.408 | Train PPL: 606.449
	 Val. Loss: 6.211 |  Val. PPL: 498.298


Started Training:   1%|          | 2/200 [02:08<3:30:27, 63.78s/it]

...

Epoch: 84 | Time: 1m 2s
	Train Loss: 0.029 | Train PPL:   1.030
	 Val. Loss: 2.136 |  Val. PPL:   8.469


Started Training:  42%|████▎     | 85/200 [1:29:11<2:00:18, 62.77s/it]

Epoch: 85 | Time: 1m 2s
	Train Loss: 0.025 | Train PPL:   1.026
	 Val. Loss: 2.149 |  Val. PPL:   8.575


=== Sample generations ===
[1] INPUT : if i could have ceased
OUTPUT: collected could have discovered <end> <end> <end> <end> acutely ached :// feels . <end> <end> omy :// <end> omy . <end> <end> <end> omy . <end> <end> <end> omy . <end> <end> <end> <end> omy . <end> <end> <end> <end> omy . <end> <end> <end> <end> omy . <end> <end>
[2] INPUT : it is quite an interesting format, if you
OUTPUT: collected quite an interesting delightful , if you need <end> <end> <end> acutely . <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> omy . <end> <end> <end> <end> <end> <end> <end>


Started Training:  43%|████▎     | 86/200 [1:30:16<2:00:27, 63.40s/it]

In [ ]:
generate_seq(model, ["Hi, introduce yourself"], bpe_tokenizer, SEQ_LEN, special_ids, gen_len=100)